In [13]:
import geopandas as gpd

df = gpd.read_file('./data/_hakone_roads.gpkg')
df = df[(df.ftCode.isin(['2701','2703','2704'])) & (df.rnkWidth != '3m未満')]
df.to_file('data/_roads_for_v.geojson', index=False, driver='GeoJSON')

In [ ]:
import json
import geopandas as gpd
import numpy as np
from pyproj import Transformer
import networkx as nx

T = Transformer.from_crs(4612, 2451, always_xy=True)

def get_edge(row):
    def get_node(coord):
        return f"{coord[0]}_{coord[1]}"

    def get_distances(coords):
        coords = list(T.itransform(coords))
        lengths = np.sqrt(np.sum(np.diff(np.array(coords), axis=0)**2, axis=1))
        return [sum(lengths[:i]) for i in range(len(lengths) + 1)]

    g = row['geometry']
    head, tail = get_node(g.coords[0]), get_node(g.coords[-1])
    distances = get_distances(g.coords)
    return [get_node(g.coords[0]), get_node(g.coords[-1]), {
        'rID': row['rID'], 
        'head': head,
        'tail': tail,
        'length': distances[-1],
        'coords': g.coords[:],
        'distances': {
            head: distances,
            tail: get_distances(reversed(g.coords))
        }
    }]

df = gpd.read_file('_roads.geojson')
g = nx.Graph()
g.add_edges_from(df.apply(get_edge, axis=1))
g = nx.MultiGraph(g)
open('_roads_nw.json', 'w').write(json.dumps(nx.node_link_data(g)))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd

matplotlib.rcParams['figure.figsize'] = (8.0, 6.0)

def draw_network(g):
    pos = {n:tuple(map(float, n.split('_'))) for n in g.nodes}
    nx.draw(g, pos, node_size=10)
    plt.show()

df = gpd.read_file('_roads.geojson')
# df.plot()
# plt.show()

draw_network(g)
